In [41]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import lightgbm as lgb

In [42]:
# Display Options of DataFrame
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", None)

In [43]:
# Loading Swap Rate Dataset
data = pd.read_csv("../cleanedData/merged.csv")

In [44]:
data['CMS Tenor'] = data['CMS Tenor'].str.removesuffix('y')
data['CMS Tenor'] = data['CMS Tenor'].astype(int)


In [45]:
# Find lag 1 by shifting the amount of data in one date

delay = len(data.loc[data['Value Date'] == "2022-09-02"])

# Realized Swap Rate Prev
data['Swap Rate Lag 1 (Real)'] = data['Swap Rate'].shift(delay)


In [46]:
data

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real)
0,2022-09-02,dummyTrade1,USD,-100,-0.0100,-227907.098775,1.0,2023-09-04,10.0,1.962246,2,0.029155,0.019155,0.0042,0.0379,0.0337,6,5,0.014955,0.018745,False,0.014955,0.177525,NaN
1,2022-09-02,dummyTrade1,USD,-50,-0.0050,-222208.400967,1.0,2023-09-04,10.0,-3.812341,2,0.029155,0.024155,0.0042,0.0379,0.0337,6,5,0.019955,0.013745,False,0.013745,0.177525,NaN
2,2022-09-02,dummyTrade1,USD,-25,-0.0025,-218960.927995,1.0,2023-09-04,10.0,4.471006,2,0.029155,0.026655,0.0042,0.0379,0.0337,6,5,0.022455,0.011245,False,0.011245,0.177525,NaN
3,2022-09-02,dummyTrade1,USD,-10,-0.0010,-216872.430106,1.0,2023-09-04,10.0,4.333398,2,0.029155,0.028155,0.0042,0.0379,0.0337,6,5,0.023955,0.009745,False,0.009745,0.177525,NaN
4,2022-09-02,dummyTrade1,USD,-5,-0.0005,-216146.310328,1.0,2023-09-04,10.0,5.679687,2,0.029155,0.028655,0.0042,0.0379,0.0337,6,5,0.024455,0.009245,False,0.009245,0.177525,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,0.0010,-547852.055930,15.0,2039-01-12,8.0,5.007952,5,0.030386,0.031386,0.0044,0.0397,0.0353,6,10,0.026986,0.008314,False,0.008314,0.253906,0.030687
3919100,2024-01-12,dummyTrade9,USD,25,0.0025,-521228.587618,15.0,2039-01-12,8.0,5.131505,5,0.030386,0.032886,0.0044,0.0397,0.0353,6,10,0.028486,0.006814,False,0.006814,0.253906,0.030687
3919101,2024-01-12,dummyTrade9,USD,50,0.0050,-464905.982097,15.0,2039-01-12,8.0,3.348622,5,0.030386,0.035386,0.0044,0.0397,0.0353,6,10,0.030986,0.004314,False,0.004314,0.253906,0.030687
3919102,2024-01-12,dummyTrade9,USD,75,0.0075,-391315.041929,15.0,2039-01-12,8.0,1.063634,5,0.030386,0.037886,0.0044,0.0397,0.0353,6,10,0.033486,0.001814,False,0.001814,0.253906,0.030687


In [47]:
zero_subset = data['Zero Rate Shock'] == 0
shock_subset = -zero_subset
zero = data[zero_subset].reset_index(drop = True)
shock = data[shock_subset].reset_index(drop = True)

In [48]:
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real)
0,2022-09-02,dummyTrade1,USD,0,0.0,-215402.656152,1.0,2023-09-04,10.0,5.016274,2,0.029155,0.029155,0.0042,0.0379,0.0337,6,5,0.024955,0.008745,False,0.008745,0.177525,NaN
1,2022-09-02,dummyTrade10,USD,0,0.0,-616299.369241,1.0,2023-09-04,10.0,-59.820414,5,0.027246,0.027246,0.0044,0.0397,0.0353,4,10,0.022846,0.012454,False,0.012454,0.177525,NaN
2,2022-09-02,dummyTrade11,USD,0,0.0,-608678.010978,1.0,2023-09-04,10.0,7.762595,10,0.027377,0.027377,0.0046,0.0417,0.0371,4,10,0.022777,0.014323,False,0.014323,0.177525,NaN
3,2022-09-02,dummyTrade12,USD,0,0.0,-609360.662412,1.0,2023-09-04,10.0,25.329779,10,0.027377,0.027377,0.0046,0.0417,0.0371,3,10,0.022777,0.014323,False,0.014323,0.177525,NaN
4,2022-09-02,dummyTrade2,USD,0,0.0,-215836.787208,1.0,2023-09-04,10.0,5.409384,2,0.029155,0.029155,0.0042,0.0379,0.0337,4,5,0.024955,0.008745,False,0.008745,0.177525,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.0,-200752.707821,15.0,2039-01-12,8.0,-0.277974,10,0.031712,0.031712,0.0046,0.0417,0.0371,4,5,0.027112,0.009988,False,0.009988,0.253906,0.031876
326588,2024-01-12,dummyTrade6,USD,0,0.0,-200966.214770,15.0,2039-01-12,8.0,-0.296353,10,0.031712,0.031712,0.0046,0.0417,0.0371,3,5,0.027112,0.009988,False,0.009988,0.253906,0.031876
326589,2024-01-12,dummyTrade7,USD,0,0.0,-507396.663276,15.0,2039-01-12,8.0,5.056103,2,0.029827,0.029827,0.0042,0.0379,0.0337,6,10,0.025627,0.008073,False,0.008073,0.253906,0.030518
326590,2024-01-12,dummyTrade8,USD,0,0.0,-508418.039331,15.0,2039-01-12,8.0,4.911479,2,0.029827,0.029827,0.0042,0.0379,0.0337,4,10,0.025627,0.008073,False,0.008073,0.253906,0.030518


In [49]:
delay = len(zero.loc[zero['Value Date'] == "2022-09-02"])
zero['Vols Prev'] = zero['Vols'].shift(delay)
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev
0,2022-09-02,dummyTrade1,USD,0,0.0,-215402.656152,1.0,2023-09-04,10.0,5.016274,2,0.029155,0.029155,0.0042,0.0379,0.0337,6,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN
1,2022-09-02,dummyTrade10,USD,0,0.0,-616299.369241,1.0,2023-09-04,10.0,-59.820414,5,0.027246,0.027246,0.0044,0.0397,0.0353,4,10,0.022846,0.012454,False,0.012454,0.177525,NaN,NaN
2,2022-09-02,dummyTrade11,USD,0,0.0,-608678.010978,1.0,2023-09-04,10.0,7.762595,10,0.027377,0.027377,0.0046,0.0417,0.0371,4,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN
3,2022-09-02,dummyTrade12,USD,0,0.0,-609360.662412,1.0,2023-09-04,10.0,25.329779,10,0.027377,0.027377,0.0046,0.0417,0.0371,3,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN
4,2022-09-02,dummyTrade2,USD,0,0.0,-215836.787208,1.0,2023-09-04,10.0,5.409384,2,0.029155,0.029155,0.0042,0.0379,0.0337,4,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.0,-200752.707821,15.0,2039-01-12,8.0,-0.277974,10,0.031712,0.031712,0.0046,0.0417,0.0371,4,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271
326588,2024-01-12,dummyTrade6,USD,0,0.0,-200966.214770,15.0,2039-01-12,8.0,-0.296353,10,0.031712,0.031712,0.0046,0.0417,0.0371,3,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271
326589,2024-01-12,dummyTrade7,USD,0,0.0,-507396.663276,15.0,2039-01-12,8.0,5.056103,2,0.029827,0.029827,0.0042,0.0379,0.0337,6,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271
326590,2024-01-12,dummyTrade8,USD,0,0.0,-508418.039331,15.0,2039-01-12,8.0,4.911479,2,0.029827,0.029827,0.0042,0.0379,0.0337,4,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271


In [50]:
zero['Vols Change'] = zero['Vols'] - zero['Vols Prev']
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change
0,2022-09-02,dummyTrade1,USD,0,0.0,-215402.656152,1.0,2023-09-04,10.0,5.016274,2,0.029155,0.029155,0.0042,0.0379,0.0337,6,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN
1,2022-09-02,dummyTrade10,USD,0,0.0,-616299.369241,1.0,2023-09-04,10.0,-59.820414,5,0.027246,0.027246,0.0044,0.0397,0.0353,4,10,0.022846,0.012454,False,0.012454,0.177525,NaN,NaN,NaN
2,2022-09-02,dummyTrade11,USD,0,0.0,-608678.010978,1.0,2023-09-04,10.0,7.762595,10,0.027377,0.027377,0.0046,0.0417,0.0371,4,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN
3,2022-09-02,dummyTrade12,USD,0,0.0,-609360.662412,1.0,2023-09-04,10.0,25.329779,10,0.027377,0.027377,0.0046,0.0417,0.0371,3,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN
4,2022-09-02,dummyTrade2,USD,0,0.0,-215836.787208,1.0,2023-09-04,10.0,5.409384,2,0.029155,0.029155,0.0042,0.0379,0.0337,4,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.0,-200752.707821,15.0,2039-01-12,8.0,-0.277974,10,0.031712,0.031712,0.0046,0.0417,0.0371,4,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636
326588,2024-01-12,dummyTrade6,USD,0,0.0,-200966.214770,15.0,2039-01-12,8.0,-0.296353,10,0.031712,0.031712,0.0046,0.0417,0.0371,3,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636
326589,2024-01-12,dummyTrade7,USD,0,0.0,-507396.663276,15.0,2039-01-12,8.0,5.056103,2,0.029827,0.029827,0.0042,0.0379,0.0337,6,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636
326590,2024-01-12,dummyTrade8,USD,0,0.0,-508418.039331,15.0,2039-01-12,8.0,4.911479,2,0.029827,0.029827,0.0042,0.0379,0.0337,4,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636


In [51]:
# Find Risk of RAN notes contributed by swaption IV changes

# change in TV caused by vega (t=0) = previous vega (t-1)* vol change
zero['Vega Lag 1']  = zero['Vega'].shift(delay)
zero['Vega Risk Exposure'] = zero['Vols Change'] * zero['Vega Lag 1']
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure
0,2022-09-02,dummyTrade1,USD,0,0.0,-215402.656152,1.0,2023-09-04,10.0,5.016274,2,0.029155,0.029155,0.0042,0.0379,0.0337,6,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN,NaN,NaN
1,2022-09-02,dummyTrade10,USD,0,0.0,-616299.369241,1.0,2023-09-04,10.0,-59.820414,5,0.027246,0.027246,0.0044,0.0397,0.0353,4,10,0.022846,0.012454,False,0.012454,0.177525,NaN,NaN,NaN,NaN,NaN
2,2022-09-02,dummyTrade11,USD,0,0.0,-608678.010978,1.0,2023-09-04,10.0,7.762595,10,0.027377,0.027377,0.0046,0.0417,0.0371,4,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN,NaN,NaN
3,2022-09-02,dummyTrade12,USD,0,0.0,-609360.662412,1.0,2023-09-04,10.0,25.329779,10,0.027377,0.027377,0.0046,0.0417,0.0371,3,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN,NaN,NaN
4,2022-09-02,dummyTrade2,USD,0,0.0,-215836.787208,1.0,2023-09-04,10.0,5.409384,2,0.029155,0.029155,0.0042,0.0379,0.0337,4,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.0,-200752.707821,15.0,2039-01-12,8.0,-0.277974,10,0.031712,0.031712,0.0046,0.0417,0.0371,4,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636,-0.156327,-0.000412
326588,2024-01-12,dummyTrade6,USD,0,0.0,-200966.214770,15.0,2039-01-12,8.0,-0.296353,10,0.031712,0.031712,0.0046,0.0417,0.0371,3,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636,-0.207006,-0.000546
326589,2024-01-12,dummyTrade7,USD,0,0.0,-507396.663276,15.0,2039-01-12,8.0,5.056103,2,0.029827,0.029827,0.0042,0.0379,0.0337,6,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636,5.049607,0.013309
326590,2024-01-12,dummyTrade8,USD,0,0.0,-508418.039331,15.0,2039-01-12,8.0,4.911479,2,0.029827,0.029827,0.0042,0.0379,0.0337,4,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636,4.263389,0.011237


In [52]:
zero['Vega Risk Exposure'].describe()

count    325620.000000
mean          0.026966
std          22.079677
min       -3300.852081
25%          -0.007612
50%           0.000000
75%           0.008292
max        5232.585440
Name: Vega Risk Exposure, dtype: float64

In [53]:
zero['Swap Rate Change'] = zero['Swap Rate'] - zero['Swap Rate Lag 1 (Real)']
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure,Swap Rate Change
0,2022-09-02,dummyTrade1,USD,0,0.0,-215402.656152,1.0,2023-09-04,10.0,5.016274,2,0.029155,0.029155,0.0042,0.0379,0.0337,6,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-09-02,dummyTrade10,USD,0,0.0,-616299.369241,1.0,2023-09-04,10.0,-59.820414,5,0.027246,0.027246,0.0044,0.0397,0.0353,4,10,0.022846,0.012454,False,0.012454,0.177525,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-09-02,dummyTrade11,USD,0,0.0,-608678.010978,1.0,2023-09-04,10.0,7.762595,10,0.027377,0.027377,0.0046,0.0417,0.0371,4,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-09-02,dummyTrade12,USD,0,0.0,-609360.662412,1.0,2023-09-04,10.0,25.329779,10,0.027377,0.027377,0.0046,0.0417,0.0371,3,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-09-02,dummyTrade2,USD,0,0.0,-215836.787208,1.0,2023-09-04,10.0,5.409384,2,0.029155,0.029155,0.0042,0.0379,0.0337,4,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.0,-200752.707821,15.0,2039-01-12,8.0,-0.277974,10,0.031712,0.031712,0.0046,0.0417,0.0371,4,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636,-0.156327,-0.000412,-0.000164
326588,2024-01-12,dummyTrade6,USD,0,0.0,-200966.214770,15.0,2039-01-12,8.0,-0.296353,10,0.031712,0.031712,0.0046,0.0417,0.0371,3,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636,-0.207006,-0.000546,-0.000164
326589,2024-01-12,dummyTrade7,USD,0,0.0,-507396.663276,15.0,2039-01-12,8.0,5.056103,2,0.029827,0.029827,0.0042,0.0379,0.0337,6,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636,5.049607,0.013309,-0.000691
326590,2024-01-12,dummyTrade8,USD,0,0.0,-508418.039331,15.0,2039-01-12,8.0,4.911479,2,0.029827,0.029827,0.0042,0.0379,0.0337,4,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636,4.263389,0.011237,-0.000691


In [54]:
zero['TV Lag 1'] = zero['TV'].shift(delay)
zero['TV Change'] = zero['TV'] - zero['TV Lag 1'] 
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure,Swap Rate Change,TV Lag 1,TV Change
0,2022-09-02,dummyTrade1,USD,0,0.0,-215402.656152,1.0,2023-09-04,10.0,5.016274,2,0.029155,0.029155,0.0042,0.0379,0.0337,6,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-09-02,dummyTrade10,USD,0,0.0,-616299.369241,1.0,2023-09-04,10.0,-59.820414,5,0.027246,0.027246,0.0044,0.0397,0.0353,4,10,0.022846,0.012454,False,0.012454,0.177525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-09-02,dummyTrade11,USD,0,0.0,-608678.010978,1.0,2023-09-04,10.0,7.762595,10,0.027377,0.027377,0.0046,0.0417,0.0371,4,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-09-02,dummyTrade12,USD,0,0.0,-609360.662412,1.0,2023-09-04,10.0,25.329779,10,0.027377,0.027377,0.0046,0.0417,0.0371,3,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-09-02,dummyTrade2,USD,0,0.0,-215836.787208,1.0,2023-09-04,10.0,5.409384,2,0.029155,0.029155,0.0042,0.0379,0.0337,4,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.0,-200752.707821,15.0,2039-01-12,8.0,-0.277974,10,0.031712,0.031712,0.0046,0.0417,0.0371,4,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636,-0.156327,-0.000412,-0.000164,-201031.681080,278.973259
326588,2024-01-12,dummyTrade6,USD,0,0.0,-200966.214770,15.0,2039-01-12,8.0,-0.296353,10,0.031712,0.031712,0.0046,0.0417,0.0371,3,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636,-0.207006,-0.000546,-0.000164,-201241.288696,275.073926
326589,2024-01-12,dummyTrade7,USD,0,0.0,-507396.663276,15.0,2039-01-12,8.0,5.056103,2,0.029827,0.029827,0.0042,0.0379,0.0337,6,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636,5.049607,0.013309,-0.000691,-509747.568328,2350.905053
326590,2024-01-12,dummyTrade8,USD,0,0.0,-508418.039331,15.0,2039-01-12,8.0,4.911479,2,0.029827,0.029827,0.0042,0.0379,0.0337,4,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636,4.263389,0.011237,-0.000691,-510774.029616,2355.990285


In [55]:
zero['TV Change'].describe()

count    325620.000000
mean        136.406926
std        9325.497424
min     -110564.166917
25%       -2034.187152
50%           1.021570
75%        2354.574763
max      181650.015378
Name: TV Change, dtype: float64

In [56]:
zero[zero['TV Change'] == zero['TV Change'].max()]

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure,Swap Rate Change,TV Lag 1,TV Change
142887,2023-04-25,dummyTrade12,USD,0,0.0,-468861.088186,1.0,2024-04-25,10.0,-28.921865,10,0.027274,0.027274,0.0046,0.0417,0.0371,3,10,0.022674,0.014426,False,0.014426,0.874463,0.028212,0.080628,0.793836,32.248660,25.600139,-0.000938,-650511.103563,181650.015378
142899,2023-04-25,dummyTrade12,USD,0,0.0,-468861.088186,1.0,2024-04-25,15.0,42.958199,10,0.027274,0.027274,0.0046,0.0417,0.0371,3,10,0.022674,0.014426,False,0.014426,0.887960,0.028212,0.109111,0.778849,-110.357855,-85.952082,-0.000938,-650511.103563,181650.015378
142911,2023-04-25,dummyTrade12,USD,0,0.0,-468861.088186,1.0,2024-04-25,1.5,0.000000,10,0.027274,0.027274,0.0046,0.0417,0.0371,3,10,0.022674,0.014426,False,0.014426,1.550981,0.028212,0.104255,1.446725,0.000000,0.000000,-0.000938,-650511.103563,181650.015378
142923,2023-04-25,dummyTrade12,USD,0,0.0,-468861.088186,1.0,2024-04-25,1.0,-0.304386,10,0.027274,0.027274,0.0046,0.0417,0.0371,3,10,0.022674,0.014426,False,0.014426,1.689528,0.028212,0.113488,1.576040,-0.191221,-0.301372,-0.000938,-650511.103563,181650.015378
142935,2023-04-25,dummyTrade12,USD,0,0.0,-468861.088186,1.0,2024-04-25,2.0,1.519547,10,0.027274,0.027274,0.0046,0.0417,0.0371,3,10,0.022674,0.014426,False,0.014426,1.418925,0.028212,0.095466,1.323459,-1.076771,-1.425063,-0.000938,-650511.103563,181650.015378
142947,2023-04-25,dummyTrade12,USD,0,0.0,-468861.088186,1.0,2024-04-25,3.0,0.527563,10,0.027274,0.027274,0.0046,0.0417,0.0371,3,10,0.022674,0.014426,False,0.014426,1.228784,0.028212,0.081732,1.147052,-0.355342,-0.407596,-0.000938,-650511.103563,181650.015378
142959,2023-04-25,dummyTrade12,USD,0,0.0,-468861.088186,1.0,2024-04-25,4.0,-0.543950,10,0.027274,0.027274,0.0046,0.0417,0.0371,3,10,0.022674,0.014426,False,0.014426,1.140051,0.028212,0.076904,1.063147,0.360520,0.383286,-0.000938,-650511.103563,181650.015378
142971,2023-04-25,dummyTrade12,USD,0,0.0,-468861.088186,1.0,2024-04-25,5.0,8.144069,10,0.027274,0.027274,0.0046,0.0417,0.0371,3,10,0.022674,0.014426,False,0.014426,1.095987,0.028212,0.077003,1.018984,0.879369,0.896062,-0.000938,-650511.103563,181650.015378
142983,2023-04-25,dummyTrade12,USD,0,0.0,-468861.088186,1.0,2024-04-25,8.0,-7.251363,10,0.027274,0.027274,0.0046,0.0417,0.0371,3,10,0.022674,0.014426,False,0.014426,0.939203,0.028212,0.077402,0.861800,21.270016,18.330509,-0.000938,-650511.103563,181650.015378
142995,2023-04-25,dummyTrade12,USD,0,0.0,-468861.088186,1.5,2024-10-25,10.0,-9.925282,10,0.027274,0.027274,0.0046,0.0417,0.0371,3,10,0.022674,0.014426,False,0.014426,0.816746,0.028212,0.078390,0.738356,15.428652,11.391843,-0.000938,-650511.103563,181650.015378


In [57]:
zero[(zero['Value Date'] == '2023-04-24') & (zero['Trade Name'] == "dummyTrade12")]

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure,Swap Rate Change,TV Lag 1,TV Change
141915,2023-04-24,dummyTrade12,USD,0,0.0,-650511.103563,1.0,2024-04-24,10.0,32.248660,10,0.028212,0.028212,0.0046,0.0417,0.0371,3,10,0.023612,0.013488,False,0.013488,0.080628,0.028732,0.082127,-0.001499,35.144771,-0.052683,-0.00052,-647405.256925,-3105.846638
141927,2023-04-24,dummyTrade12,USD,0,0.0,-650511.103563,1.0,2024-04-24,15.0,-110.357855,10,0.028212,0.028212,0.0046,0.0417,0.0371,3,10,0.023612,0.013488,False,0.013488,0.109111,0.028732,0.110075,-0.000964,-132.994530,0.128188,-0.00052,-647405.256925,-3105.846638
141939,2023-04-24,dummyTrade12,USD,0,0.0,-650511.103563,1.0,2024-04-24,1.5,0.000000,10,0.028212,0.028212,0.0046,0.0417,0.0371,3,10,0.023612,0.013488,False,0.013488,0.104255,0.028732,0.107858,-0.003603,0.000000,-0.000000,-0.00052,-647405.256925,-3105.846638
141951,2023-04-24,dummyTrade12,USD,0,0.0,-650511.103563,1.0,2024-04-24,1.0,-0.191221,10,0.028212,0.028212,0.0046,0.0417,0.0371,3,10,0.023612,0.013488,False,0.013488,0.113488,0.028732,0.118366,-0.004879,-0.160528,0.000783,-0.00052,-647405.256925,-3105.846638
141963,2023-04-24,dummyTrade12,USD,0,0.0,-650511.103563,1.0,2024-04-24,2.0,-1.076771,10,0.028212,0.028212,0.0046,0.0417,0.0371,3,10,0.023612,0.013488,False,0.013488,0.095466,0.028732,0.098847,-0.003381,-1.072497,0.003626,-0.00052,-647405.256925,-3105.846638
141975,2023-04-24,dummyTrade12,USD,0,0.0,-650511.103563,1.0,2024-04-24,3.0,-0.355342,10,0.028212,0.028212,0.0046,0.0417,0.0371,3,10,0.023612,0.013488,False,0.013488,0.081732,0.028732,0.084235,-0.002503,-0.124524,0.000312,-0.00052,-647405.256925,-3105.846638
141987,2023-04-24,dummyTrade12,USD,0,0.0,-650511.103563,1.0,2024-04-24,4.0,0.360520,10,0.028212,0.028212,0.0046,0.0417,0.0371,3,10,0.023612,0.013488,False,0.013488,0.076904,0.028732,0.079641,-0.002737,-0.622510,0.001704,-0.00052,-647405.256925,-3105.846638
141999,2023-04-24,dummyTrade12,USD,0,0.0,-650511.103563,1.0,2024-04-24,5.0,0.879369,10,0.028212,0.028212,0.0046,0.0417,0.0371,3,10,0.023612,0.013488,False,0.013488,0.077003,0.028732,0.079500,-0.002497,2.289082,-0.005715,-0.00052,-647405.256925,-3105.846638
142011,2023-04-24,dummyTrade12,USD,0,0.0,-650511.103563,1.0,2024-04-24,8.0,21.270016,10,0.028212,0.028212,0.0046,0.0417,0.0371,3,10,0.023612,0.013488,False,0.013488,0.077402,0.028732,0.079219,-0.001817,26.882856,-0.048841,-0.00052,-647405.256925,-3105.846638
142023,2023-04-24,dummyTrade12,USD,0,0.0,-650511.103563,1.5,2024-10-24,10.0,15.428652,10,0.028212,0.028212,0.0046,0.0417,0.0371,3,10,0.023612,0.013488,False,0.013488,0.078390,0.028732,0.079460,-0.001070,13.343729,-0.014275,-0.00052,-647405.256925,-3105.846638
